The following code fetches the last 500 traffic entries for a given street alongside the corresponding weather. Then it creates a prompt and feeds it to a llama3 model by Ollama on a virtual machine. It stores the prompts and responses to S3 and adds them back to our database in a new table 'traffic_prediction'.

In [ ]:
import json
import os
import requests
import boto3
import time
import psycopg2
import re
from psycopg2 import sql
from decimal import Decimal

def get_filtered_traffic_weather_data(street_name):
    query = sql.SQL("""
    SELECT t.jam_factor,
           w.temperature,
           w.description AS weather_description,
           to_char(to_timestamp(t.timestamp), 'YYYY-MM-DD HH24:MI:SS') AS traffic_datetime
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (ORDER BY t.timestamp DESC) AS row_num
        FROM public.traffic_data t
        WHERE t.description LIKE {street_name}
          AND EXTRACT(HOUR FROM to_timestamp(t.timestamp)) NOT BETWEEN 0 AND 4
    ) AS t
    JOIN (
        SELECT *,
               ROW_NUMBER() OVER (ORDER BY w.timestamp DESC) AS row_num
        FROM public.weather_data w
    ) AS w ON t.row_num = w.row_num
    WHERE t.row_num % 4 = 0
    LIMIT 500;
    """).format(street_name=sql.Literal('%' + street_name + '%'))
    return query

def fetch_filtered_traffic_weather_data(street_name):
    conn = None
    results = []
    try:
        conn = psycopg2.connect(
            database=os.environ['DB_NAME'],
            user=os.environ['USERNAME'],
            password=os.environ['PASSWORD'],
            host=os.environ['ENDPOINT'],
            port='5432'
        )
        print("Database connection established.")
        cursor = conn.cursor()
        sql_query = get_filtered_traffic_weather_data(street_name)

        # Execute the SQL query
        cursor.execute(sql_query)

        # Fetch the results
        results = cursor.fetchall()

        # Close the cursor and connection
        cursor.close()
        print("Data fetched successfully.")

    except psycopg2.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

    return results

def convert_decimal(obj):
    if isinstance(obj, Decimal):
        return float(obj)
    raise TypeError

def create_table_if_not_exists():
    conn = None
    try:
        conn = psycopg2.connect(
            database=os.environ['DB_NAME'],
            user=os.environ['USERNAME'],
            password=os.environ['PASSWORD'],
            host=os.environ['ENDPOINT'],
            port='5432'
        )
        print("Database connection established for table creation check.")
        cursor = conn.cursor()

        # Create table if it does not exist
        create_table_query = """
        CREATE TABLE IF NOT EXISTS public.traffic_prediction (
            id SERIAL PRIMARY KEY,
            prompt TEXT NOT NULL,
            response TEXT NOT NULL,
            timestamp TIMESTAMP NOT NULL
        );
        """
        cursor.execute(create_table_query)

        # Commit the transaction
        conn.commit()

        # Close the cursor and connection
        cursor.close()
        print("Table creation check completed.")

    except psycopg2.Error as e:
        print(f"Database error during table creation check: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed after table creation check.")

def save_prediction_to_db(prompt, response_text):
    create_table_if_not_exists()  # Ensure the table exists before saving the prediction

    conn = None
    try:
        conn = psycopg2.connect(
            database=os.environ['DB_NAME'],
            user=os.environ['USERNAME'],
            password=os.environ['PASSWORD'],
            host=os.environ['ENDPOINT'],
            port='5432'
        )
        print("Database connection established for saving prediction.")
        cursor = conn.cursor()

        # Insert the prompt and response into the traffic_prediction table
        insert_query = """
        INSERT INTO public.traffic_prediction (prompt, response, timestamp)
        VALUES (%s, %s, to_timestamp(%s))
        """
        current_time = time.time()
        cursor.execute(insert_query, (prompt, response_text, current_time))

        # Commit the transaction
        conn.commit()

        # Close the cursor and connection
        cursor.close()
        print("Prediction saved to database successfully.")

    except psycopg2.Error as e:
        print(f"Database error while saving prediction: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed after saving prediction.")

def lambda_handler(event, context):
    # Example street name; in practice, this can be passed in via the event or context.
    street_name = event.get('street_name', 'Bahnhofplatz')

    # Fetch the data
    data = fetch_filtered_traffic_weather_data(street_name)

    # Convert the data to a list of strings
    formatted_data = [', '.join(map(str, row)) for row in data]

    # Save the filtered data to an S3 file
    try:
        s3 = boto3.client('s3')
        bucket_name = "lakebucketv3"
        file_content = '\n'.join(formatted_data)
        file_name = f"filtered_data_{time.time()}.txt"  # Add timestamp to make filenames unique
        s3.put_object(
            Body=file_content.encode('utf-8'),
            Bucket=bucket_name,
            Key=file_name
        )
        print("Filtered data file uploaded to S3.")
    except boto3.exceptions.ClientError as e:
        print(f"Error uploading filtered data file to S3: {e}")

    # prediction conditions
    temperature = 0  # Celsius
    weather_description = 'sunny'  # change

    # Add the specified text to the beginning of the output
    first_prompt = f"You are given the following information for one street in the city of Zurich. It has a jamfactor, the temperature in degrees (C) and the weather status. The jamfactor can take a value from 0 to 10 and represents how jammed the street is, 10 meaning the road is closed. Make a prediction for the jam factor of the next day based on the following weather: {temperature}C {weather_description}. Don't analyze anything else about the data, just predict the new jam factor."

    # Construct the prompt by combining the first prompt and the formatted data
    prompt = '\n'.join(formatted_data + [first_prompt])

    # Prepare the payload for the API request
    payload = json.dumps({
      "model": "llama3",
      "prompt": prompt,
      "stream": False
    })

    # Send the API request
    url = "http://86.119.48.155:11434/api/generate"
    headers = {
      'Content-Type': 'application/json'
    }
    response = requests.post(url, headers=headers, data=payload)

    # Save the output of the llama3 model to an S3 file
    try:
        file_content = response.text
        file_name = f"llama3_output_{time.time()}.txt"  # Add timestamp to make filenames unique
        s3.put_object(
            Body=file_content.encode('utf-8'),
            Bucket=bucket_name,
            Key=file_name
        )
        print("Llama3 output file uploaded to S3.")
    except boto3.exceptions.ClientError as e:
        print(f"Error uploading Llama3 output file to S3: {e}")

    # Save the prompt and response to the database
    save_prediction_to_db(prompt, response.text)

    # Return the response
    return {
        'statusCode': response.status_code,
        'body': response.text
    }
